# Demo of the Adaptive Ground Point Filtering Library

This notebook gives a tour of the capabilities of `filteradapt`. It is continuously updated while we develop the library. You can execute the code in each cell by pressing `Ctrl+Enter`. To start your own project based on `filteradapt`, you can either edit this notebook or start a new one.

The first thing to do in a Jupyter notebook that uses `filteradapt` is to import the library:

In [ ]:
%load_ext lab_black

In [ ]:
import filteradapt
import pdal

The next step you will typically do is to construct a dataset object by reading a LAS/LAZ file:

In [ ]:
ds = filteradapt.DataSet("data/500k_NZ20_Westport.laz")
ds.show()

The data set can be visualized in the notebook through its `show()` method:

In [ ]:
from filteradapt.paths import locate_file
import json

filename = locate_file("data/500k_NZ20_Westport.laz")


def setup_csf(filename):
    print(filename)
    my_json = {
        "pipeline": [
            filename,
            {"type": "filters.csf"},
            {"type": "filters.range", "limits": "Classification[2:2]"},
        ]
    }
    return json.dumps(my_json)


def setup_nothing(filename):
    print(filename)
    my_json = {"pipeline": [filename]}
    return json.dumps(my_json)

In [ ]:
test = setup_nothing(filename)
pipeline = pdal.Pipeline(test)
pipeline.validate()
%time n_points = pipeline.execute()
print("Pipeline selected {} points ".format(n_points))

In [ ]:
test2 = setup_csf(filename)
pipeline2 = pdal.Pipeline(test2)
pipeline2.validate()
%time n_points = pipeline2.execute()
print("Pipeline selected {} points ".format(n_points))

In [ ]:
import pandas as pd

# Load Pipeline output in python objects
arr = pipeline2.arrays[0]
description = arr.dtype.descr
cols = [col for col, __ in description]
df = pd.DataFrame({col: arr[col] for col in cols})
df["X_0"] = df["X"]
df["Y_0"] = df["Y"]
df["Z_0"] = df["Z"]
df["X"] = df["X"] - df["X_0"].min()
df["Y"] = df["Y"] - df["Y_0"].min()
df["Z"] = df["Z"] - df["Z_0"].min()

In [ ]:
import ipyvolume.pylab as p3

fig = p3.figure(width=1000)
fig.xlabel = "Y"
fig.ylabel = "Z"
fig.zlabel = "X"
all_points = p3.scatter(df["Y"], df["Z"], df["X"], color="red", size=0.05)
p3.squarelim()
p3.show()

In [ ]:
print(len(df))
df.head()